## Workflow Fundamentals

In [2]:
:dep candle-core = "0.9.1"
:dep candle-nn = "0.9.1"
:dep plotters = { version = "^0.3.0", default-features = false, features = ["evcxr", "all_series"] }

In [3]:
use candle_core::{DType, Device, Module, ModuleT, NdArray, Tensor, Var, D};
use candle_nn::linear::{Linear, linear};
use candle_nn::loss::mse;
use candle_nn::var_builder::{VarBuilder, SimpleBackend};
use candle_nn::{Optimizer, VarMap, SGD};
extern crate plotters;
use plotters::prelude::*;
use std::iter::{zip};


In [4]:
use candle_core::{Device, Tensor, DType};
let device = Device::Cpu;


### Step 1.  Getting Data Ready

Let's create some data that can be used for linear regression.

One interesting change from the original
```python
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias
```

Because the tensor operationsl return Result<Tensor>, you have unwrap results of the multiplication before adding the bias.

A way (there are probably better ways), to print the first ten items is to turn the tensor into a vector using once of the methods on the Tensor.

In [ ]:
let device = Device::Cpu;
let weight = 0.7;
let bias = 0.3;

let start = 0.0;
let end = 1.0;
let step = 0.02;
let X = Tensor::arange_step(start, end, step, &device)?;
//Do the multiplication first
let y = ((weight * X.clone())? + bias)?;

let x_vec = X.clone().to_vec1::<f64>()?;
let y_vec = y.clone().to_vec1::<f64>()?;
println!("{:?}", &x_vec[..10]);
println!("{:?}", &y_vec[..10]);
let train_split: usize = (0.8 * x_vec.len() as f32) as usize;
println!("Train: {}, Test:{}\n", train_split, x_vec.len() - train_split as usize);

let X_train = Tensor::from_slice(&x_vec[..train_split], (train_split,1), &device)?;
let y_train = Tensor::from_slice(&y_vec[..train_split], (train_split, 1), &device)?;
let x_test = Tensor::from_slice(&x_vec[train_split..x_vec.len()], (x_vec.len() - train_split, 1), &device)?;
let y_test = Tensor::from_slice(&y_vec[train_split..y_vec.len()], (y_vec.len() - train_split, 1), &device)?;


[0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12000000000000001, 0.14, 0.16, 0.18]
[0.3, 0.314, 0.32799999999999996, 0.34199999999999997, 0.356, 0.37, 0.384, 0.398, 0.412, 0.426]
Train: 40, Test:10



### Visualizing the Data

For this we will use plotter-rs.  Candle Tensors can be converted to vecs of different dimensions, so creating data points should be pretty easy

In [6]:
let training_datapoints: Vec<(f64, f64)> = zip(X_train.squeeze(1)?.to_vec1::<f64>()?, y_train.squeeze(1)?.to_vec1::<f64>()?).collect();
let test_datapoints: Vec<(f64, f64)> = zip(x_test.squeeze(1)?.to_vec1::<f64>()?, y_test.squeeze(1)?.to_vec1::<f64>()?).collect();


In [7]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Training Data", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..1f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    chart.draw_series(training_datapoints
                      .iter()
                      .map(|(x,y)| Circle::new((*x, *y), 2, BLUE.filled())))?
                      .label("Training Data")
                      .legend(|(x,y)| Circle::new((x, y), 2, BLUE.filled()));; 
    
    chart.draw_series(test_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, GREEN.filled())))?
                  .label("Test Data")
                  .legend(|(x,y)| Circle::new((x, y), 2, GREEN.filled()));; 

    chart.configure_series_labels().position(SeriesLabelPosition::UpperLeft).border_style(BLACK).background_style(WHITE.mix(0.1)).draw()?;
    
    Ok(())
}).style("width: 60%")

Training Data
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Training Data
 
 
Test Data

### 2.  Build Model

Create a linear regression model using Candle's NN linear layer.  You can let candle initialize a linear layer using the ```linear``` function, which automatically setup a weight and bias, or you can use ```Linear::new``` and provide the weight and bias yourself.  

In this case we'll create the weight and bias ourselves since there are iterations of the notebook where the weights are randomly initialized such that they are already very close to the original data.  This way we can initialize it with something that is way off so we can see the loss decrease over each training epoch.

**Note**

You'll want to use Var to declare your initial weights and bias and then clone them throughout the cells of your notebook.

In [8]:
let w = Var::new(&[[-0.2439f64]], &Device::Cpu)?;
let b = Var::new(0.3170f64, &Device::Cpu)?;

let model = Linear::new(w.as_tensor().clone(), Some(b.as_tensor().clone()));

In [ ]:
//Let's look at the initialized weight and bias
println!("Weight: {}", model.weight());
match model.clone().bias() {
    Some(b) => println!("Bias: {}", b),
    None => {}
};


Weight: [[-0.2439]]
Tensor[[1, 1], f64]
Bias: [0.3170]


Let's have the model make some predications without any training

In [10]:
let y_preds = model.forward(&x_test)?;

Tensor[[], f64]


In [11]:
let pred_datapoints: Vec<(f64, f64)> = zip(x_test.squeeze(1)?.to_vec1::<f64>()?, y_preds.squeeze(1)?.to_vec1::<f64>()?).collect();

In [12]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Predication with No Training", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..1f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    chart.draw_series(training_datapoints
                      .iter()
                      .map(|(x,y)| Circle::new((*x, *y), 2, BLUE.filled())))?
                      .label("Training Data")
                      .legend(|(x,y)| Circle::new((x, y), 2, BLUE.filled()));; 
    
    chart.draw_series(test_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, GREEN.filled())))?
                  .label("Test Data")
                  .legend(|(x,y)| Circle::new((x, y), 2, GREEN.filled()));; 

        chart.draw_series(pred_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, RED.filled())))?
                  .label("Predication")
                  .legend(|(x,y)| Circle::new((x, y), 2, RED.filled()));; 

    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .border_style(BLACK)
        .background_style(WHITE.mix(0.1))
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Predication with No Training
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Training Data
 
 
Test Data
 
 
Predication

Let's train the model now and see what happens

In [13]:
let mut train_loss_values: Vec<f64> = vec![];
let mut test_loss_values: Vec<f64> = vec![];
let mut epoch_count: Vec<f64> = vec![];
//Initialize the optimizer with the model weights and biases; be sure to clone
let mut opt = SGD::new(vec![w.clone(), b.clone()], 0.01)?;

for epoch in 0..5000 {
    //1.  Forward pass on the training data
    let y_preds = model.forward(&X_train)?;

    //2.  Calculate the loss, for this we will use Mean Squared Error (mse) from candle
    let loss = mse(&y_preds, &y_train)?;
    
    //3.  Candle does not have a zero_grad mechanism
    //4.  Get a GradStore from the loss
    let bp = loss.backward()?;
    //5. progress the optimzer
    opt.step(&bp);

    //Test the model
    let test_pred = model.forward(&x_test)?;
    let test_loss = mse(&test_pred, &y_test)?;
    
    if epoch % 10 == 0 {
        epoch_count.push(epoch as f64);
        train_loss_values.push(loss.to_scalar::<f64>()?);
        test_loss_values.push(test_loss.to_scalar::<f64>()?);
    }
}


()

In [14]:
// println!("Epochs: {:?}", epoch_count);
// println!("Training Loss Values: {:?}", train_loss_values);
// println!("Test Loss Values: {:?}", test_loss_values);

let train_loss_datapoints: Vec<(f64, f64)> = zip(epoch_count.clone(), train_loss_values.clone()).collect();
let test_loss_datapoints: Vec<(f64, f64)> = zip(epoch_count.clone(), test_loss_values.clone()).collect();
//println!("Train Loss Data Points: {:?}", train_loss_datapoints);
let max_tl: f64 = *test_loss_values.first().unwrap() + 0.05;
let max_epoch: f64 = *epoch_count.last().unwrap();


In [15]:


evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Training and Test Loss curves", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..max_epoch, 0f64..max_tl)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    //chart.draw_series(LineSeries::new(x_values.into_iter().map(|x| (x, 0.3 * x)), BLACK)).unwrap();
    chart.draw_series(LineSeries::new(train_loss_datapoints.clone().into_iter().map(|(x,y)| (x, y)), BLUE))
        .unwrap()
        .label("Training Loss")
        .legend(|(x,y)| Rectangle::new([(x - 10, y + 1), (x, y)], BLUE));
    chart.draw_series(LineSeries::new(test_loss_datapoints.clone().into_iter().map(|(x,y)| (x, y)), RED))
        .unwrap()
        .label("Test Loss")
        .legend(|(x,y)| Rectangle::new([(x - 10, y + 1), (x, y)], RED));

    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperRight)
        .border_style(BLACK)
        .background_style(WHITE.mix(0.1))
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Training and Test Loss curves
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
 
0.0
 
 
 
500.0
 
 
 
1000.0
 
 
 
1500.0
 
 
 
2000.0
 
 
 
2500.0
 
 
 
3000.0
 
 
 
3500.0
 
 
 
4000.0
 
 
 
4500.0
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="47,314 48,367 49,384 50,396 51,403 52,408 54,411 55,413 56,414 57,415 58,416 59,416 61,417 62,417 63,418 64,418 65,418 66,418 68,419 69,419 70,419 71,419 72,420 73,420 75,420 76,420 77,421 78,421 79,421 80,421 82,421 83,422 84,422 85,422 86,422 88,422 89,423 90,423 91,423 92,423 93,423 95,423 96,424 97,424 98,424 99,424 100,424 102,424 103,425 104,425 105,425 106,425 107,425 109,425 110,425 111,426 112,426 113,426 114,426 116,426 117,426 118,426 119,426 120,426 122,427 123,427 124,427 125,427 126,427 127,427 129,427 130,427 131,427 132,427 133,428 134,428 136,428 137,428 138,428 139,428 140,428 141,428 143,428 144,428 145,428 146,428 147,429 148,429 150,429 151,429 152,429 153,429 154,429 156,429 157,429 158,429 159,429 160,429 161,429 163,429 164,429 165,429 166,430 167,430 168,430 170,430 171,430 172,430 173,430 174,430 175,430 177,430 178,430 179,430 180,430 181,430 182,430 184,430 185,430 186,430 187,430 188,430 190,430 191,430 192,431 193,431 194,431 195,431 197,431 198,431 199,431 200,431 201,431 202,431 204,431 205,431 206,431 207,431 208,431 209,431 211,431 212,431 213,431 214,431 215,431 216,431 218,431 219,431 220,431 221,431 222,431 224,431 225,431 226,431 227,431 228,431 229,431 231,431 232,431 233,431 234,431 235,432 236,432 238,432 239,432 240,432 241,432 242,432 243,432 245,432 246,432 247,432 248,432 249,432 250,432 252,432 253,432 254,432 255,432 256,432 258,432 259,432 260,432 261,432 262,432 263,432 265,432 266,432 267,432 268,432 269,432 270,432 272,432 273,432 274,432 275,432 276,432 277,432 279,432 280,432 281,432 282,432 283,432 284,432 286,432 287,432 288,432 289,432 290,432 292,432 293,432 294,432 295,432 296,432 297,432 299,432 300,432 301,432 302,432 303,432 304,432 306,432 307,432 308,432 309,432 310,432 311,432 313,432 314,432 315,432 316,432 317,432 318,432 320,432 321,432 322,432 323,432 324,432 326,432 327,432 328,432 329,432 330,432 331,432 333,432 334,432 335,432 336,432 337,432 338,432 340,432 341,432 342,432 343,432 344,432 345,432 347,432 348,432 349,432 350,432 351,432 352,432 354,432 355,432 356,432 357,432 358,432 360,432 361,432 362,432 363,432 364,432 365,432 367,432 368,432 369,432 370,432 371,432 372,432 374,432 375,432 376,432 377,432 378,432 379,432 381,432 382,432 383,432 384,432 385,432 386,432 388,432 389,432 390,432 391,432 392,432 394,432 395,432 396,432 397,432 398,432 399,432 401,432 402,432 403,432 404,432 405,432 406,432 408,432 409,432 410,432 411,432 412,432 413,432 415,432 416,432 417,432 418,432 419,432 420,432 422,432 423,432 424,432 425,432 426,432 428,432 429,432 430,432 431,432 432,432 433,432 435,432 436,432 437,432 438,432 439,432 440,432 442,432 443,432 444,432 445,432 446,432 447,432 449,432 450,432 451,432 452,432 453,432 454,432 456,432 457,432 458,432 459,432 460,432 462,432 463,432 464,432 465,432 466,432 467,432 469,432 470,432 471,432 472,432 473,432 474,432 476,432 477,432 478,432 479,432 480,432 481,432 483,432 484,432 485,432 486,432 487,432 488,432 490,432 491,432 492,432 493,432 494,432 496,432 497,432 498,432 499,432 500,432 501,432 503,432 504,432 505,432 506,432 507,432 508,432 510,432 511,432 512,432 513,432 514,432 515,432 517,432 518,432 519,432 520,432 521,432 522,432 524,432 525,432 526,432 527,432 528,432 530,432 531,432 532,432 533,432 534,432 535,432 537,432 538,432 539,432 540,432 541,432 542,432 544,432 545,432 546,432 547,432 548,432 549,432 551,432 552,432 553,432 554,432 555,432 556,432 558,432 559,432 560,432 561,432 562,432 564,432 565,432 566,432 567,432 568,432 569,432 571,432 572,432 573,432 574,432 575,432 576,432 578,432 579,432 580,432 581,432 582,432 583,432 585,432 586,432 587,432 588,432 589,432 590,432 592,432 593,432 594,432 595,

Let's see how the model is doing after 5000 epochs

In [16]:
let y_preds_after_training = model.forward(&x_test)?;

In [17]:
let pred_after_training_datapoints: Vec<(f64, f64)> = zip(x_test.squeeze(1)?.to_vec1::<f64>()?, y_preds_after_training.squeeze(1)?.to_vec1::<f64>()?).collect();

In [20]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Predication with 5000 Epochs", ("Arial", 20).into_font())
        .margin(7)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..1f64, 0f64..1f64)?;
    
    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .draw()?;

    chart.draw_series(training_datapoints
                      .iter()
                      .map(|(x,y)| Circle::new((*x, *y), 2, BLUE.filled())))?
                      .label("Training Data")
                      .legend(|(x,y)| Circle::new((x, y), 2, BLUE.filled()));; 
    
    chart.draw_series(test_datapoints
                  .iter()
                  .map(|(x,y)| Circle::new((*x, *y), 2, GREEN.filled())))?
                  .label("Test Data")
                  .legend(|(x,y)| Circle::new((x, y), 2, GREEN.filled()));; 

    chart.draw_series(pred_datapoints
              .iter()
              .map(|(x,y)| Circle::new((*x, *y), 2, RED.filled())))?
              .label("Original Predication")
              .legend(|(x,y)| Circle::new((x, y), 2, RED.filled()));; 

    chart.draw_series(pred_after_training_datapoints
          .iter()
          .map(|(x,y)| Circle::new((*x, *y), 2, BLACK.filled())))?
          .label("Predication Post Training")
          .legend(|(x,y)| Circle::new((x, y), 2, BLACK.filled()));; 

    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .border_style(BLACK)
        .background_style(WHITE.mix(0.1))
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Predication with 5000 Epochs
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Training Data
 
 
Test Data
 
 
Original Predication
 
 
Predication Post Training

In [19]:
println!("Weight: {}", model.weight());
match model.clone().bias() {
    Some(b) => println!("Bias: {}", b),
    None => {}
};


Weight: [[0.6928]]
Tensor[[1, 1], f64]
Bias: [0.3030]
Tensor[[], f64]
